# Imports

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import openai
from openai.types.chat import ChatCompletion
from openai import OpenAI
import time
import pandas as pd

# API key

In [ ]:
api_key = epi_key

In [ ]:
client = OpenAI(
    api_key=api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o-mini",
)

In [ ]:
chat_completion

ChatCompletion(id='chatcmpl-AdHLDKwdhAgoLQ2wffHYKBSZRMb8S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test. How can I assist you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733924815, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_bba3c8e70b', usage=CompletionUsage(completion_tokens=12, prompt_tokens=12, total_tokens=24, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

# Testing definitions for final prompt design

In [ ]:
def definition(text):
    # Make the API call
    definition = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
              "role": "system",
              "content": "You are a scientific analytic tool designed to identify affect heuristic, availability heuristic, representativeness heuristic, confirmation bias and all-or-nothing thinking in text."
            },
            {
                "role": "user",
                "content": f"Please provide a definition of '{text}' in two sentences.",
            }
        ]
    )

    # Extract the sentiment from the response
    result = definition.choices[0].message.content.strip()
    return result

# Example usage
bias = "all-or-nothing thinking"
result = definition(bias)
print(result)

All-or-nothing thinking is a cognitive distortion that involves viewing situations in black-and-white terms, with no middle ground. This type of thinking often leads to extreme evaluations of experiences, where outcomes are seen as completely successful or entirely failed, disregarding any nuances or complexities.


# Automatic analysis

In [ ]:
def create_messages(text):
    return [
        {
            "role": "system",
            "content": """"content": "You are a text annotation tool designed to identify and quantify biases and heuristics in a given text. Your role is to evaluate each statement and assign probabilities (0 to 1) for the presence of the following biases or heuristics : affect heuristic, availability heuristic, representativeness heuristic, confirmation bias, and all-or-nothing thinking. The given text can have more than 1 bias or heuristic. If the text does not include any heuristic or bias, or if it is ambiguous, respond with 'No_bias' or 'Unclear', respectively.
            Definitions:

            - Affect Heuristic: When one's emotions or feelings influence their judgments and decisions, with positive emotions increasing perceived benefits and reducing risks, and negative emotions doing the opposite

            - Availability Heuristic: When one overestimates the probability of events they can easily recall or have recently experienced, and underestimates the likelihood of less memorable or less frequently encountered events.

            - Representativeness Heuristic: When one judges an object or person based on how closely they resemble the typical category member, either assuming they belong to that category or do not belong based on this similarity.

            - Confirmation Bias: When one seeks out or favors information supporting their beliefs while disregarding or ignoring evidence that contradicts them.

            - All-or-Nothing Thinking: When one views situations in extreme, black-and-white terms, without recognizing any middle ground or nuance.


            Instructions:

            1. Carefully analyze the main idea and emotional tone of the given text.

            2. Assign a probability (0-1) using three decimal places for each bias, reflecting how likely the bias is present in the text.

            3. Respond with a flat string containing probabilities for each bias in this format:

            "affect_h": 0.336, "availability_h": 0.284, "representativeness_h": 0.207, "confirmation_b": 0.083, "all_or_nothing": 0.164, "no_bias": 0.07, "unclear": 0.02.


            Consider the broader context, definitions, and cues such as:

            Affect Heuristic: Emotional language.

            Availability Heuristic: References to recent or vivid events.

            Representativeness Heuristic: Stereotypes or generalizations.

            Confirmation Bias: Selective use of supporting information.

            All-or-Nothing Thinking: Extreme terms like "always," "never," "only." Determine the probabilities based on definitions."""
                    },
        {
            "role": "user",
            "content": f"Analyze the following text and return probabilities for each heuristic or bias: '{text}'"
        },
        {
            "role": "assistant",
            "content": """Based on the analysis, does the speaker use heuristics or cognitive biases? Assign a probability (0-1) for each bias, reflecting how likely the bias is present in the text based on the definitions in the format: affect_h": 0.732, "availability_h": 0.312, "representativeness_h": 0.008, "confirmation_b": 0.578, "all_or_nothing": 0.164, "no_bias": 0.07, "unclear": 0.02"""
        }
    ]

In [ ]:
def analyze_sentences_batch(sentences, batch_size):
    results = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        batch_messages = [create_messages(sentence) for sentence in batch]
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=sum(batch_messages, []),
                temperature=0.2
                #max_tokens=100
            )

            for choice in response.choices:
                pathos = choice.message.content.strip().lower()
                results.append(pathos)
        except openai.error.RateLimitError as e:
            print(f"Rate limit reached: {e}. Waiting 60 seconds.")
            time.sleep(300)
            continue
        except openai.error.OpenAIError as e:
            print(f"Failed to get response from OpenAI: {e}")
            results.extend([None] * len(batch))
            continue
    return results

In [ ]:
# Function to process a single file
def process_file(input_file, output_file, checkpoint_file, batch_size):
    if os.path.exists(output_file):
        print(f"Skipping {input_file} as it is already processed.")
        return

    df = pd.read_excel(input_file)
    start_index = 0

    # Check if checkpoint exists
    if os.path.exists(checkpoint_file):
        checkpoint_df = pd.read_pickle(checkpoint_file)
        start_index = checkpoint_df.shape[0]
        print(f"Resuming {input_file} from sentence {start_index}.")
        df = df[start_index:]

    # Analyze sentences
    for index, row in enumerate(df.itertuples(), start=start_index):
        pathos_results = analyze_sentences_batch([row.Text], batch_size)
        df.at[index, 'bias'] = pathos_results[0] if pathos_results else None

        if index % 10 == 0:
            # Save checkpoint
            checkpoint_df = df.iloc[:index + 1]
            checkpoint_df.to_pickle(checkpoint_file)
            print(f"Checkpoint saved at sentence {index} for {input_file}.")

    # Save final results
    df.to_excel(output_file, index=False)
    print(f"Analysis completed for {input_file} and saved as {output_file}")

    # Remove checkpoint file
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)

In [ ]:
dir = "/content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/"
input_file = dir + 'Zgodność_s2_twitter_AI.xlsx'
output_file = dir + 'Twitter_AI_with_unclear_output.xlsx'
checkpoint_file = dir + 'Twitter_AI_checkpoint.pkl'

batch_size = 1093

# Process the file
process_file(input_file, output_file, checkpoint_file, batch_size)

Checkpoint saved at sentence 0 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 10 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 20 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 30 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 40 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 50 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do zgodności/Zgodność_s2_twitter_AI.xlsx.
Checkpoint saved at sentence 60 for /content/drive/MyDrive/MA thesis/Open AI automatic annotation/Pliki do 